In [1]:
import pandas as pd
df=pd.read_csv("medquad.csv")
df.drop(columns=['source','focus_area'])
df.to_csv('cleaned_data.csv')

In [2]:
import os
key = os.environ['GOOGLE_API_KEY']

In [3]:
from langchain.llms import GooglePalm

/Users/subahnsh47/Desktop/palm/.venv/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [4]:
llm=GooglePalm(google_api_key=key,temperature=0.9)

/Users/subahnsh47/Desktop/palm/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
poem=llm('write a poem in 4 lines on apple')
print(poem)

**Apple**

Round and red, a taste of heaven,
A symbol of love and health,
Nature's gift, the apple.


In [6]:
from langchain.vectorstores import FAISS
from langchain.embeddings import GooglePalmEmbeddings
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.chains import RetrievalQA

In [7]:
loader=CSVLoader(file_path='cleaned_data.csv',source_column='question',encoding='ISO-8859-1')
data=loader.load()

In [8]:
embeddings=GooglePalmEmbeddings(model_name="models/embedding-gecko-001")
# vectordb=embeddings.embed_query(data)

In [9]:
# from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.embeddings import HuggingFaceEmbeddings

# Initialize instructor embeddings using the Hugging Face model
embeddings = HuggingFaceEmbeddings()

e = embeddings.embed_query("What is your refund policy?")

In [10]:
vectordb=FAISS.from_documents(documents=data,embedding=embeddings)

In [11]:
vectordb.save_local("faiss_index")

In [12]:
retriever = vectordb.as_retriever()

In [13]:
from langchain.prompts import PromptTemplate

prompt_template = """Given the following context and a question, generate an answer based on this context only.
In the answer try to provide as much text as possible from "response" section in the source document context without making much changes.
If the answer is not found in the context, kindly state "I don't know." Don't try to make up an answer.

CONTEXT: {context}

QUESTION: {question}"""


PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)
chain_type_kwargs = {"prompt": PROMPT}

In [14]:
chain=RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    input_key='query',
    chain_type_kwargs=chain_type_kwargs,
    return_source_documents=True
)

In [15]:
def ask(query):
    results = chain(query)
    if results:
        print("You Asked")
        print(results['query'])
        print("\n")
        
        print("Your Answer is")
        print(results['result'])
        print("\n")
        
        
        print("Answer is found from -")
        print(results['source_documents'])
        print("\n")
    else:
        print("No results found.")


In [17]:
ask('What is High Blood Pressure')

Retrying langchain.llms.google_palm.generate_with_retry.<locals>._generate_with_retry in 2.0 seconds as it raised ServiceUnavailable: 503 DNS resolution failed for generativelanguage.googleapis.com:443: C-ares status is not ARES_SUCCESS qtype=A name=generativelanguage.googleapis.com is_balancer=0: DNS query cancelled.
Retrying langchain.llms.google_palm.generate_with_retry.<locals>._generate_with_retry in 4.0 seconds as it raised ServiceUnavailable: 503 DNS resolution failed for generativelanguage.googleapis.com:443: C-ares status is not ARES_SUCCESS qtype=A name=generativelanguage.googleapis.com is_balancer=0: DNS query cancelled.
Retrying langchain.llms.google_palm.generate_with_retry.<locals>._generate_with_retry in 8.0 seconds as it raised ServiceUnavailable: 503 DNS resolution failed for generativelanguage.googleapis.com:443: C-ares status is not ARES_SUCCESS qtype=A name=generativelanguage.googleapis.com is_balancer=0: DNS query cancelled.
Retrying langchain.llms.google_palm.gene

In [1]:
ask('What is Breast Cancer?')

NameError: name 'ask' is not defined